In [1]:
# !pip install transformers
#!pip install pandas==1.3.0
#!pip install sklearn

Reviews Dataset: <a href="https://www.kaggle.com/volodymyrgavrysh/imdb-sentiment-10k-reviews-binary-classification">https://www.kaggle.com/volodymyrgavrysh/imdb-sentiment-10k-reviews-binary-classification</a>

In [2]:
import pandas as pd
#print(pd.__version__)
import re
import torch
print(torch.__version__)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForMaskedLM
  
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

1.10.0


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
twitter=pd.read_csv('training.1600000.processed.noemoticon.csv',encoding='latin1',header=None)

In [4]:
twitter.head()
twitter=twitter[[0,5]]

In [7]:
twitter.head()
twitter[5]=twitter[5].apply(preprocess)

In [8]:
twitter.head()

,0,5
0,0,"Awww, thats a bummer. You shoulda got Davi..."
1,0,is upset that he cant update his Facebook by t...
2,0,I dived many times for the ball. Managed to s...
3,0,my whole body feels itchy and like its on fire
4,0,"no, its not behaving at all. im mad. why am i..."


# Loading Dataset File

In [9]:
#df=pd.read_csv('imdb_10K_sentimnets_reviews.csv')

In [10]:
#df.head()

# Split Dataset for training and testing

In [11]:
#training_dataset=df.sample(frac=0.7)
#test_dataset=df.drop(training_dataset.index)
training_dataset=twitter.sample(frac=0.7)
test_dataset=twitter.drop(training_dataset.index)

In [12]:
training_dataset.head()

,0,5
1258794,4,is planning stuff for this week...
307991,0,are you dating anyone? ALSO YES I DID kou is ...
483812,0,I know how you feel my ipod decided to stop ...
1270289,4,kickin it at s house with everyone. cool cro...
90648,0,no! I never found my selma! oh well! May...


In [13]:
print(len(training_dataset))
print(len(test_dataset))

1120000
480000


In [14]:
training_dataset['review'][6800]

KeyError: 'review'

In [6]:
regexMap={r"<[\w'/'\s]*>": "",r"[\'\"\-]+": "",r"@[\w]+":"",r"http\S+" : ""}
def preprocess(datainput):
    t=datainput
    for regx in regexMap.keys():
        t = re.sub(regx, regexMap[regx], t)
    return t


In [ ]:
training_dataset['review']=training_dataset['review'].apply(preprocess)
test_dataset['review']=test_dataset['review'].apply(preprocess)

In [ ]:
training_dataset['review'][6800]

In [ ]:
vectorizer = CountVectorizer()

# Fit Count Vectorizer on training data

In [ ]:
vectorizer.fit(training_dataset['review'].values)

In [ ]:
vectorizer.vocabulary_

## Stopwords

In [ ]:
stop_words_vectorizer=CountVectorizer(stop_words='english')
stop_words_vectorizer.fit(training_dataset['review'].values)


In [ ]:
stop_words_vectorizer.vocabulary_

In [ ]:
x_input=stop_words_vectorizer.transform(training_dataset['review'].values)
x_test_input=stop_words_vectorizer.transform(test_dataset['review'].values)

In [ ]:
nb = MultinomialNB()

In [ ]:
nb.fit(x_input,training_dataset['sentiment'])

In [ ]:
y_pred_class = nb.predict(x_test_input)
metrics.accuracy_score(test_dataset["sentiment"].values, y_pred_class)

In [ ]:
print(classification_report(y_true=test_dataset["sentiment"].values,y_pred=y_pred_class))

In [ ]:
len(training_dataset[(training_dataset['sentiment']==1)])

In [ ]:
df_copy=test_dataset.copy()
df_copy['sentiment']=y_pred_class

In [ ]:
df_copy.head()
df_copy[df_copy['sentiment']==1]

In [ ]:
df_copy['review'][22]